In [1]:
import pandas as pd
import numpy as np
import random

# 1. Parametreler
n_patients = 1000
np.random.seed(42) # Tekrarlanabilirlik için

# 2. Temel Değişkenler
ids = [f"PT-{i:04d}" for i in range(n_patients)]
ages = np.random.normal(60, 12, n_patients).astype(int) # Ort 60 yaş, SS 12
genders = np.random.choice(['Male', 'Female'], n_patients, p=[0.55, 0.45])
treatment_group = np.random.choice(['Standard_Chemo', 'Immunotherapy_X'], n_patients, p=[0.5, 0.5])

# 3. Biyobelirteçler (Gerçekçi Korelasyonlar)
# İmmünoterapi alanlarda bazı yan etkiler (Enzim yüksekliği) daha fazla olsun
alt_levels = []
tumor_size_baseline = []

for group in treatment_group:
    if group == 'Standard_Chemo':
        alt_levels.append(np.random.normal(30, 10)) # Normal karaciğer enzimi
        tumor_size_baseline.append(np.random.normal(4.5, 1.5)) # cm cinsinden
    else:
        alt_levels.append(np.random.normal(45, 20)) # İlaç biraz karaciğeri yoruyor (Gürültü)
        tumor_size_baseline.append(np.random.normal(4.6, 1.6)) # Başlangıç benzer

# 4. Hedef Değişken: Survival Time (Ay cinsinden) ve Event (Ölüm:1, Yaşıyor:0)
survival_months = []
events = []

for i in range(n_patients):
    # Temel risk faktörleri
    base_survival = 36 # 3 yıl baz yaşam
    
    # Yaş etkisi (Yaşlılar daha riskli)
    age_effect = (ages[i] - 60) * -0.5 
    
    # Tümör etkisi (Büyük tümör riski artırır)
    tumor_effect = (tumor_size_baseline[i] - 4.5) * -4
    
    # Tedavi Etkisi (İmmünoterapi ömrü uzatsın)
    treatment_effect = 12 if treatment_group[i] == 'Immunotherapy_X' else 0
    
    # Rastgelelik (Biyolojik varyasyon)
    noise = np.random.normal(0, 8)
    
    final_survival = base_survival + age_effect + tumor_effect + treatment_effect + noise
    final_survival = max(1, final_survival) # Negatif olamaz
    
    # Censoring (Sansürleme) Mekanizması
    # Çalışma 48 ay sürdü. Eğer hasta 50 ay yaşayacaksa, biz onu 48. ayda "yaşıyor" (0) olarak görürüz.
    study_duration = 48
    if final_survival > study_duration:
        observed_time = study_duration
        event_status = 0 # Censored (Alive at end of study)
    else:
        observed_time = final_survival
        event_status = 1 # Event occurred (Death)
        
    survival_months.append(round(observed_time, 1))
    events.append(event_status)

# 5. DataFrame Oluşturma
df_oncology = pd.DataFrame({
    'Patient_ID': ids,
    'Age': ages,
    'Gender': genders,
    'Treatment_Group': treatment_group,
    'Baseline_Tumor_Size_cm': np.round(tumor_size_baseline, 2),
    'ALT_Enzyme_Level': np.round(alt_levels, 1),
    'Survival_Months': survival_months,
    'Event_Status': events # 1: Dead, 0: Censored
})

# CSV'ye kaydetme simülasyonu
# df_oncology.to_csv("oncology_survival_data.csv", index=False)
print("Onkoloji Verisi Hazır:")
print(df_oncology.head())
print("\nDağılım:\n", df_oncology['Event_Status'].value_counts())

Onkoloji Verisi Hazır:
  Patient_ID  Age  Gender  Treatment_Group  Baseline_Tumor_Size_cm  \
0    PT-0000   65    Male   Standard_Chemo                    4.82   
1    PT-0001   58    Male   Standard_Chemo                    3.76   
2    PT-0002   67  Female   Standard_Chemo                    5.43   
3    PT-0003   78  Female   Standard_Chemo                    2.72   
4    PT-0004   57    Male  Immunotherapy_X                    4.30   

   ALT_Enzyme_Level  Survival_Months  Event_Status  
0              23.9             30.0             1  
1              42.0             48.0             0  
2              11.2             32.2             1  
3              23.6             44.6             1  
4              32.5             48.0             0  

Dağılım:
 Event_Status
1    678
0    322
Name: count, dtype: int64


In [2]:
df_oncology.head()

,Patient_ID,Age,Gender,Treatment_Group,Baseline_Tumor_Size_cm,ALT_Enzyme_Level,Survival_Months,Event_Status
0,PT-0000,65,Male,Standard_Chemo,4.82,23.9,30.0,1
1,PT-0001,58,Male,Standard_Chemo,3.76,42.0,48.0,0
2,PT-0002,67,Female,Standard_Chemo,5.43,11.2,32.2,1
3,PT-0003,78,Female,Standard_Chemo,2.72,23.6,44.6,1
4,PT-0004,57,Male,Immunotherapy_X,4.30,32.5,48.0,0
